In [13]:
import numpy as np
import pandas as pd
import scipy.constants as constants
from scipy.integrate import simps, quad
from scipy.interpolate import splrep, splint
from scipy.optimize import fmin

# plotting
import panel as pn
import param
import hvplot.pandas
from io import StringIO

In [2]:
def calculate_SQ(Tcell=300., bandgap=1.63):
    
    h = constants.physical_constants['Planck constant'][0] # units of J*s
    h_ev = constants.physical_constants['Planck constant in eV s'][0]
    c_nm = (constants.physical_constants['speed of light in vacuum'][0]) * 1e9
    c = (constants.physical_constants['speed of light in vacuum'][0])

    e_charge = constants.physical_constants['elementary charge'][0] 
    kb_ev = constants.physical_constants['Boltzmann constant in eV/K'][0]

    bandgap_array_min = 0.5 #in eV
    bandgap_array_max = 3 # in eV
    num_points_bandgap_array = 30


    """Programming below"""
    bandgap_array = np.linspace(bandgap_array_min, bandgap_array_max, num_points_bandgap_array)
    #First convert AM1.5 spectrum from W/m^2/nm to W/m^2/ev

#     if self.ui.load_checkBox.isChecked():
#         astmg173 = np.loadtxt(self.astmg173_file, delimiter = ',', skiprows = 2)
#         am15_wav = np.copy(astmg173[:,0]) #AM1.5 wavelength axis in nm
#         am15 = np.copy(astmg173[:,2]) #AM1.5 in units of W/m^2/nm = J/s*m^2/nm

    astmg173 = np.loadtxt(r"ASTMG173.csv", delimiter=",", skiprows=2)
    am15_wav = np.copy(astmg173[:,0]) #AM1.5 wavelength axis in nm
    am15 = np.copy(astmg173[:,1]) #AM1.5 in units of W/m^2/nm = J/s*m^2/nm

    total_power_nm = simps(am15, x = am15_wav) #Integrate over nm to check that total power density = 1000 W/m^2


    am15_ev = h_ev * (c_nm) / (am15_wav )
    am15_wats_ev = am15 * (h_ev * c_nm/ ((am15_ev) ** 2.0))

    am15_ev_flip = am15_ev[::-1] 
    am15_wats_ev_flip = am15_wats_ev[::-1]


    total_power_ev = simps(am15_wats_ev_flip, x = am15_ev_flip) #Integrate over eV to check that total power density = 1000 W/m^2


    am15_photons_ev  = am15_wats_ev_flip / (am15_ev_flip * e_charge)

    am15_photons_nm = am15 / (am15_ev * e_charge)

    total_photonflux_ev = simps(am15_photons_ev, x = am15_ev_flip)

    total_photonflux_nm = simps(am15_photons_nm , x = am15_wav)

    total_photonflux_ev_splrep = splrep(am15_ev_flip, am15_photons_ev)

    emin = am15_ev_flip[0]
    emax = am15_ev_flip[len(am15_ev_flip) - 1]
    

    def solar_photons_above_gap(Egap): #units of photons / sec *m^2
        return splint(Egap, emax,total_photonflux_ev_splrep) 


    def RR0(Egap):
        integrand = lambda eV : eV ** 2.0 / (np.exp(eV / (kb_ev * Tcell)) - 1)
        integral = quad(integrand, Egap, emax, full_output=1)[0]
        return ((2.0 * np.pi / ((c ** 2.0) * (h_ev ** 3.0)))) * integral

    
    def current_density(V, Egap): #to get from units of amps / m^2 to mA/ cm^2 ---multiply by 1000 to convert to mA ---- multiply by (0.01 ^2) to convert to cm^2
        cur_dens =  e_charge * (solar_photons_above_gap(Egap) - RR0(Egap) * np.exp( V / (kb_ev * Tcell)))    
        return cur_dens * 1000 * (0.01 ** 2.0)
    
    
    def JSC(Egap): 
        return current_density(0, Egap) 

    
    def VOC(Egap):
        return (kb_ev * Tcell) * np.log(solar_photons_above_gap(Egap) / RR0(Egap))


    def fmax(func_to_maximize, initial_guess=0):
        """return the x that maximizes func_to_maximize(x)"""
        func_to_minimize = lambda x : -func_to_maximize(x)
        return fmin(func_to_minimize, initial_guess, disp=False)[0]    

    
    def V_mpp_Jmpp_maxpower_maxeff_ff(Egap):

        vmpp = fmax(lambda V : V * current_density(V, Egap))    
        jmpp = current_density(vmpp, Egap)

        maxpower =  vmpp * jmpp
        max_eff = maxpower / (total_power_ev * 1000 * (0.01 ** 2.0))
        jsc_return =  JSC(Egap)
        voc_return = VOC(Egap)
        ff = maxpower / (jsc_return * voc_return)    
        return [vmpp, jmpp, maxpower, max_eff, ff, jsc_return, voc_return]


    maxpcemeta = V_mpp_Jmpp_maxpower_maxeff_ff(bandgap)

#     print('For Bandgap = %.3f eV, TCell = %.3f K:\nJSC = %.3f mA/cm^2\nVOC = %.3f V\nFF = %.3f\nPCE = %.3f' % (bandgap, Tcell, maxpcemeta[5], maxpcemeta[6],maxpcemeta[4], maxpcemeta[3] * 100)))


    pce_array = np.empty_like(bandgap_array)
    ff_array = np.empty_like(bandgap_array)
    voc_array = np.empty_like(bandgap_array)
    jsc_array = np.empty_like(bandgap_array)
    
    for i in range(len(bandgap_array)):
        metadata = V_mpp_Jmpp_maxpower_maxeff_ff(bandgap_array[i])
        pce_array[i] = metadata[3] 
        ff_array[i] = metadata[4]
        voc_array[i] = metadata[6]
        jsc_array[i] = metadata[5]

    out_array = np.array((bandgap_array,100*pce_array,ff_array, voc_array,jsc_array)).T
    
    df = pd.DataFrame(out_array, columns=["Bandgap (eV)", "PCE (%)", "Fill Factor", "Voc (V)", "Jsc (mA/cm2)"])

    
#     fig, axes = plt.subplots(nrows=2, ncols=2)    
#     fig.suptitle('Tcell = %.3f K' %(Tcell))
    
#     axes.flat[0].set_xlim(bandgap_array[0], bandgap_array[len(bandgap_array) - 1])
#     axes.flat[0].set_ylabel('PCE (%)')
#     axes.flat[0].set_xlabel('Bandgap (eV)')
#     axes.flat[0].plot(bandgap_array, pce_array * 100)
    
#     axes.flat[1].set_ylim(0, 1)
#     axes.flat[1].set_xlim(bandgap_array[0], bandgap_array[len(bandgap_array) - 1])
#     axes.flat[1].set_ylabel('Fill Factor')
#     axes.flat[1].set_xlabel('Bandgap (eV)')
#     axes.flat[1].plot(bandgap_array, ff_array)
    
#     axes.flat[2].set_xlim(bandgap_array[0], bandgap_array[len(bandgap_array) - 1])
#     axes.flat[2].set_ylabel('Jsc (mA/cm$^2$)')
#     axes.flat[2].set_xlabel('Bandgap (eV)')
#     axes.flat[2].plot(bandgap_array, jsc_array)
    
#     axes.flat[3].set_xlim(bandgap_array[0], bandgap_array[len(bandgap_array) - 1])
#     axes.flat[3].set_ylabel('Voc (V)')
#     axes.flat[3].set_xlabel('Bandgap (eV)')
#     axes.flat[3].plot(bandgap_array, voc_array, label = 'S-Q Voc')
#     axes.flat[3].plot(bandgap_array, bandgap_array, '--', label = 'Bandgap')
#     axes.flat[3].legend(loc="best")


    def JV_curve(Egap):
        volt_array = np.linspace(0, VOC(Egap), 200)
        j_array = np.empty_like(volt_array)
        for i in range(len(volt_array)):
            j_array[i] = current_density(volt_array[i], Egap)
        return [volt_array, j_array]


    jv_meta = JV_curve(bandgap)
    v_array = jv_meta[0]
    jv_array = jv_meta[1]
    
    df_2 = pd.DataFrame(np.array([jv_meta[0], -jv_meta[1]]).T, columns=["Voltage (V)", "Current Density (mA/cm2)"])

#     fig2, ax = plt.subplots()
#     ax.set_ylabel('Current Density (mA/cm$^2$)')
#     ax.set_xlabel('Voltage (V)')
#     ax.plot(v_array, -jv_array)
#     ax.set_title(f"J-V Curve for {bandgap} eV")
    
    return df, df_2

In [55]:
class ShockleyQueisserLimit(param.Parameterized):
    
    Temperature  = param.Number(default=300., bounds=(100., 500.))
    Bandgap = param.Number(default=1.63, bounds=(0.4, 3.0))
    

    def plot_all_vs_all_bandgap(self):
        
        self.df_1 = calculate_SQ(Tcell=self.Temperature)[0]
        
#         self.sio_1 = StringIO()
#         self.df_1.to_csv(self.sio_1)
#         self.sio_1.seek(0)

        return self.df_1.hvplot.line(x="Bandgap (eV)",
                              y=["PCE (%)", "Fill Factor", "Voc (V)", "Jsc (mA/cm2)"],
                              xlabel="Bandgap (eV)",
                              subplots=True,
                              shared_axes=False,
                              alpha=0.5,
                              width=350,
                              height=350,
                              grid=True
                              ).cols(2)
    
    
    def plot_J_V_curve_for_single_bandgap(self):
    
        self.df_2 = calculate_SQ(Tcell=self.Temperature, bandgap=self.Bandgap)[1]
        
#         self.sio_2 = StringIO()
#         self.df_2.to_csv(self.sio_2)
#         self.sio_2.seek(0)
        
        return self.df_2.hvplot.line(x="Voltage (V)",
                              y="Current Density (mA/cm2)",
                              title=f"J-V Curve for {self.Bandgap:.2f} eV",
                              xlabel="Voltage (V)",
                              ylabel="Current Density (mA/cm2)",
                              alpha=0.5,
                              width=550,
                              height=450,
                              grid=True
                             )
    

obj = ShockleyQueisserLimit()
obj

ShockleyQueisserLimit(Bandgap=1.63, Temperature=300.0, name='ShockleyQueisserLimit39832')

In [56]:
text = """## Shockley-Queisser Limit Calculator\n

Change the Temperature and Bandgap to update the S-Q limit and plots. 
You can hover over the plots to see individual values, zoom-in/out and also save the plots."""

pn.Row(
    pn.Column(
        text,
        obj.param.Temperature,
        obj.param.Bandgap,
        obj.plot_J_V_curve_for_single_bandgap),
    
    obj.plot_all_vs_all_bandgap).servable();

Row
    [0] Column
        [0] Markdown(str)
        [1] FloatSlider(end=500.0, name='Temperature', start=100.0, value=300.0)
        [2] FloatSlider(end=3.0, name='Bandgap', start=0.4, value=1.63)
        [3] ParamMethod(method)
    [1] ParamMethod(method)